In [ ]:
import pandas as pd
import numpy as np
import sklearn
import xgboost as xgb

# Exploratory Data Analysis

In [7]:
ls -l data/

total 142568
-rw-rw-rw-@ 1 pavlemedvidovic  staff     56067 Oct  6  2024 item.csv
-rw-rw-rw-@ 1 pavlemedvidovic  staff  19547042 Oct  6  2024 promotion.csv
-rw-rw-rw-@ 1 pavlemedvidovic  staff  53376614 Oct  6  2024 sales.csv
-rw-rw-rw-@ 1 pavlemedvidovic  staff      4177 Oct  6  2024 supermarkets.csv


In [11]:
items = pd.read_csv('data/item.csv')
items.head()

,code,descrption,type,brand,size
0,3000005040,AUNT JEM ORIGINAL PANCAKE MIX,Type 1,Aunt Jemima,2 LB
1,3000005070,A/JEM COMPLETE PANCAKE MI,Type 1,Aunt Jemima,32 OZ
2,3000005300,AJ BUTTERMILK PANCAKE MIX,Type 1,Aunt Jemima,32 OZ
3,3000005350,A J BTRMLK COMP PNCK MIX,Type 1,Aunt Jemima,1 LB
4,1600015760,BC PANCAKE MIX BUTTERMILK,Type 1,Bisquick,6.75 OZ


In [17]:
# Count distinct values in each column
items.nunique()

code          927
descrption    872
type            4
brand         131
size          146
dtype: int64

In [13]:
promo = pd.read_csv('data/promotion.csv')
promo.head()

,code,supermarkets,week,feature,display,province
0,2700042240,285,91,Not on Feature,Mid-Aisle End Cap,2
1,2700042292,285,92,Interior Page Feature,Not on Display,2
2,2700042274,285,92,Interior Page Feature,Not on Display,2
3,2700042273,285,92,Interior Page Feature,Not on Display,2
4,2700042254,285,92,Interior Page Feature,Not on Display,2


In [18]:
promo.nunique()

code            479
supermarkets    381
week             62
feature           8
display          11
province          2
dtype: int64

In [14]:
sales = pd.read_csv('data/sales.csv')
sales.head()

,code,amount,units,time,province,week,customerId,supermarket,basket,day,voucher
0,7680850106,0.80,1,1100,2,1,125434,244,1,1,0
1,3620000470,3.59,1,1100,2,1,125434,244,1,1,0
2,1800028064,2.25,1,1137,2,1,108320,244,2,1,0
3,9999985067,0.85,1,1148,2,1,162016,244,3,1,0
4,9999985131,2.19,1,1323,2,1,89437,244,4,1,0


In [29]:
# Get minimum and maximum customerId
sales['customerId'].min(), sales['customerId'].max()

(np.int64(1), np.int64(510027))

In [24]:
sales.nunique()

code              782
amount            971
units              30
time             1440
province            2
week               25
customerId     249610
supermarket       376
basket         665450
day               166
voucher             2
dtype: int64

In [16]:
supermarkets = pd.read_csv('data/supermarkets.csv')
supermarkets.head()

,supermarket_No,postal-code
0,199,30319
1,200,30134
2,201,30066
3,202,31093
4,203,30542


In [20]:
supermarkets.nunique()

supermarket_No    387
postal-code       299
dtype: int64

# Data Cleaning
Data is not in a great format for our purposes
1. Encode categorical variables (brands, types, features, etc.) to not be text
2. ~~Convert weights (this might be more trouble than it's worth)~~
3. Are joins needed?
Upon further inspection, I may not need as much cleaning as I thought

In [ ]:
# Encode categorical variables
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()


# Collaborative Filtering
Generate a sparse matrix where each row is a user and each column represents an item. The value in each column will be the number of times each user bought each item. This is normally done with ratings, but I will use number of times purchased as a rating proxy.

In [ ]:
# Create collaborative filtering matrix
# Pivot sales data to create a user-item matrix
# Rows are customerId, columns are item codes, values are number of units purchased
user_item_matrix = sales.pivot_table(index='customerId', columns='code', values='units', aggfunc='sum', fill_value=0)
user_item_matrix.head()

code,111112360,566300023,566300028,566300029,566300035,601011292,601011293,601011294,601011295,601011296,...,9999985134,9999985137,9999985165,9999985215,9999985216,9999985217,9999985260,9999985261,9999985488,9999985766
customerId,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Get user item matrix entries where 111112360 is greater than 0 (just to sanity check that the matrix is correct)
user_item_matrix[user_item_matrix[111112360] > 0]

code,111112360,566300023,566300028,566300029,566300035,601011292,601011293,601011294,601011295,601011296,...,9999985134,9999985137,9999985165,9999985215,9999985216,9999985217,9999985260,9999985261,9999985488,9999985766
customerId,,,,,,,,,,,,,,,,,,,,,
175058,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
197660,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
215056,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
215652,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
221275,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
364919,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
user_item_matrix.shape

(249610, 782)

In [34]:
# Find similar items using cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
item_similarity = cosine_similarity(user_item_matrix.T)
item_similarity_df = pd.DataFrame(item_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns)
item_similarity_df.head()

code,111112360,566300023,566300028,566300029,566300035,601011292,601011293,601011294,601011295,601011296,...,9999985134,9999985137,9999985165,9999985215,9999985216,9999985217,9999985260,9999985261,9999985488,9999985766
code,,,,,,,,,,,,,,,,,,,,,
111112360,1.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
566300023,0.0,1.000000,0.035322,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.001720,0.000000,0.0,0.0,0.006492,0.008158,0.000000,0.0
566300028,0.0,0.035322,1.000000,0.0,0.011358,0.002872,0.0,0.0,0.0,0.000573,...,0.002773,0.0,0.001357,0.000949,0.0,0.0,0.003841,0.012259,0.002558,0.0
566300029,0.0,0.000000,0.000000,1.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
566300035,0.0,0.000000,0.011358,0.0,1.000000,0.007025,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.003373,0.000000,0.0


In [35]:
# Get most commonly bought items together with item 111112360
item_111112360_similar = item_similarity_df[111112360].sort_values(ascending=False)
item_111112360_similar.head(10)

code
111112360     1.000000
1488650001    0.344265
6971911131    0.152145
7065070052    0.040071
7680852180    0.031898
7130000020    0.029620
3800007266    0.023388
7520000980    0.014098
3000005560    0.012332
3620000432    0.011681
Name: 111112360, dtype: float64